In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely import geometry
import seaborn as sns
import ternary
from toolbox import hillshade
import rasterio as rio
%matplotlib inline

import xarray as xr

import glob
import os
import pandas as pd

In [2]:
def plot_corr(dataframe, size=10):
    plt.figure(figsize = (size+size*0.2, size), facecolor='w')
    corrMatrix = dataframe.corr()
    sns.heatmap(np.round(corrMatrix,2), annot=True)
    plt.xticks(rotation=90, ha='right')
    plt.yticks(rotation=0, ha='right')

In [3]:
def compute_pca(df, variables, explained_variance='on'):
    
    import warnings
    warnings.filterwarnings('ignore')

    import numpy as np
    import matplotlib.pyplot as plt
    from sklearn.decomposition import PCA
    from sklearn.preprocessing import StandardScaler
    
    X = df.loc[:, variables].values
    mu_ = np.mean(X, axis=0)
    sd_ = np.std(X, axis=0)
    scaler = StandardScaler()
    x = scaler.fit_transform(X)
    
    n_components = len(variables)
    pca = PCA(n_components=n_components, whiten=True)
    pca.fit(x)
    
    n_pcs = np.arange(1, len(variables)+1)
    pc = ['PC']*(len(variables) + 1)

    PCs = []
    for i, k in zip(pc, n_pcs):
        PCs.append(i+str(k))
        
    PC_transformed = []
    for i in range(len(variables)):
        pc_dot = np.dot(pca.transform(x)[:,:i+1], pca.components_[:i+1,:])
        pc_dot_ = sd_*pc_dot + mu_
        PC_transformed.append(pc_dot_)
    
    pcs = np.array(PC_transformed)
        
    fig, ax=plt.subplots(figsize=(len(variables), len(variables)))
    im=ax.matshow(pca.components_,cmap='plasma')
    for (i, j), z in np.ndenumerate(pca.components_):
        ax.text(j, i, '{:0.2f}'.format(z), ha='center', va='center')
    fig.colorbar(im, label='Eigenvalues')
    plt.gca().set_aspect('equal')
    plt.gcf().set_size_inches(10, 8)
    plt.yticks(range(len(variables)), PCs,fontsize=9)
    plt.xticks(range(len(variables)), variables,rotation=55,fontsize = 10, ha='left')
    plt.tight_layout()
    plt.show()
    
    if explained_variance == 'on':
        plt.figure(figsize=(8, 4), facecolor='w')
        plt.bar(PCs,pca.explained_variance_ratio_ *100, color='orangered')
        plt.xticks(rotation = 90)
        plt.ylabel('Variance explained by component (%)')
        plt.grid(which='both', linewidth=0.4)
    
    return pcs

### Importing XYZ remote sensing dataset

In [12]:
MDT = pd.read_csv('/home/ggrl/usp/4-ano/remote-sensing/tf/csv/segundo/test.csv')
MDT= MDT.drop('Unnamed: 3',axis=1)
#df_1= df_1.drop('Z',axis=1)

In [13]:
MDT

,X,Y,MDT
0,398336.153139,9.502136e+06,301.0
1,398366.926714,9.502136e+06,299.0
2,398397.700289,9.502136e+06,300.0
3,398428.473864,9.502136e+06,300.0
4,398459.247439,9.502136e+06,300.0
...,...,...,...
446035,421447.108058,9.484010e+06,470.0
446036,421477.881634,9.484010e+06,474.0
446037,421508.655209,9.484010e+06,478.0
446038,421539.428784,9.484010e+06,480.0


In [14]:
# the path to your csv file directory
mycsvdir = '/home/ggrl/usp/4-ano/remote-sensing/tf/csv/segundo'

# get all the csv files in that directory (assuming they have the extension .csv)
csvfiles = glob.glob(os.path.join(mycsvdir, '*.csv'))
for csvfile in csvfiles:
    print(csvfile,end='\n')

/home/ggrl/usp/4-ano/remote-sensing/tf/csv/segundo/pontosstq_AST_13-05-2004.csv
/home/ggrl/usp/4-ano/remote-sensing/tf/csv/segundo/pontossqt_LC08_Level_2_20-09-2022.csv
/home/ggrl/usp/4-ano/remote-sensing/tf/csv/segundo/pontossqt_gama_ternario.csv
/home/ggrl/usp/4-ano/remote-sensing/tf/csv/segundo/test.csv
/home/ggrl/usp/4-ano/remote-sensing/tf/csv/segundo/pontossqt_magnetometria_tilt_e_derivada_vertical.csv
/home/ggrl/usp/4-ano/remote-sensing/tf/csv/segundo/pontossqt_LC08_Level_2_08-02-2019.csv


In [15]:
# loop through the files and read them in with pandas
dataframes = []  # a list to hold all the individual pandas DataFrames

for csvfile in csvfiles:
    df = pd.read_csv(csvfile)
    print(df.head())
    dataframes.append(df)

     MDT  _1  _2   _3  _4  _5  _6  _7  _8  _9   _10   _11   _12   _13   _14
0  301.0  71  42  100  61  32  34  30  24  20  1225  1337  1442  1688  1760
1  299.0  77  43  109  62  34  36  31  25  20  1225  1337  1442  1688  1760
2  300.0  72  42   99  64  33  36  32  26  22  1232  1350  1451  1698  1773
3  300.0  69  40   95  62  33  36  31  26  19  1232  1350  1451  1698  1773
4  300.0  71  40  104  61  33  34  30  26  21  1232  1350  1451  1698  1773
     MDT    _1    _2    _3     _4     _5     _6     _7     _8
0  301.0  8580  9047  9792  10884  14749  19507  14810  50174
1  299.0  8567  9027  9833  11040  15272  19590  14836  50227
2  300.0  8550  9001  9781  10958  15157  20128  15179  50243
3  300.0  8561  8995  9703  10828  14711  19496  14830  50252
4  300.0  8610  9018  9771  10807  14632  19157  14719  50252
     MDT   _1  _2  _3
0  301.0  128  48  24
1  299.0  136  48  16
2  300.0  136  48  16
3  300.0  136  48  16
4  300.0  144  48  16
               X             Y    MDT  U

In [16]:
result = pd.concat(dataframes, ignore_index=True,axis=1)
result
#result.to_csv('/home/ggrl/usp/4-ano/remote-sensing/tf/csv/all.csv')

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,301.0,71,42,100,61,32,34,30,24,20,...,81,301.0,8266,8316,9813,8560,20922,15306,10883,45775
1,299.0,77,43,109,62,34,36,31,25,20,...,98,299.0,8326,8414,9824,8721,20703,15579,11100,45809
2,300.0,72,42,99,64,33,36,32,26,22,...,113,300.0,8328,8411,9791,8728,19463,15596,11182,45836
3,300.0,69,40,95,62,33,36,31,26,19,...,127,300.0,8277,8359,9653,8654,19398,15290,11009,45827
4,300.0,71,40,104,61,33,34,30,26,21,...,132,300.0,8311,8383,9748,8649,20212,15361,11009,45804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446035,470.0,216,165,145,118,94,106,102,102,73,...,156,470.0,8164,8294,9596,8771,19254,15060,11010,46010
446036,474.0,192,147,134,102,82,86,88,82,56,...,147,474.0,8092,8172,9479,8478,20301,14589,10560,45966
446037,478.0,229,177,148,90,70,77,72,70,51,...,136,478.0,8049,8178,9552,8522,20100,14531,10529,46012
446038,480.0,166,123,118,87,65,72,72,65,46,...,122,480.0,8087,8204,9630,8534,20389,14776,10660,46013


In [9]:
#df_result = pd.concat([df_1,result], axis=1)

NameError: name 'df_1' is not defined

In [32]:
geotiff_path ='/home/ggrl/usp/4-ano/remote-sensing/tf/tf_dados/tif/AST/stq_AST_13-05-2004.tif'
geotiff_ds = xr.open_dataset(geotiff_path,engine='rasterio')
geotiff_ds

Warning 1: TIFFReadDirectory:Sum of Photometric type-related color channels and ExtraSamples doesn't match SamplesPerPixel. Defining non-color channels as ExtraSamples.


<xarray.Dataset>
Dimensions:      (band: 14, x: 1602, y: 1260)
Coordinates:
  * band         (band) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * x            (x) float64 3.979e+05 3.979e+05 ... 4.219e+05 4.219e+05
  * y            (y) float64 9.503e+06 9.503e+06 ... 9.484e+06 9.484e+06
    spatial_ref  int64 ...
Data variables:
    band_data    (band, y, x) float32 ...

In [29]:
#geotiff_ds = geotiff_da.to_dataset('band')

AttributeError: 'Dataset' object has no attribute 'to_dataset'

In [33]:
geotiff_df = geotiff_ds.to_dataframe()

In [34]:
#geotiff_df.head()
geotiff_df = geotiff_df.reset_index()

In [35]:
df_rs =geotiff_df

df_rs = df_rs.rename(columns={1:"VNIR_Band1",
                      2:"VNIR_Band2",
                      3:"VNIR_Band3N",
                      4:"SWIR_Band4",
                      5:"SWIR_Band5",
                      6:"SWIR_Band6",
                      7:"SWIR_Band7",
                      8:"SWIR_Band8",
                      9:"SWIR_Band9",
                      10:"TIR_Band10",
                      11:"TIR_Band11",
                      12:"TIR_Band1",
                      13:"TIR_Band13",
                      14:"TIR_Band14"})

df_ASTER = df_rs.rename(columns={'x':'E_utm','y':'N_utm'})
df_ASTER.head()

,band,E_utm,N_utm,spatial_ref,band_data
0,1,397852.5,9502517.5,0,0.0
1,1,397852.5,9502502.5,0,0.0
2,1,397852.5,9502487.5,0,0.0
3,1,397852.5,9502472.5,0,0.0
4,1,397852.5,9502457.5,0,0.0


In [36]:
df_ASTER.head()

,band,E_utm,N_utm,spatial_ref,band_data
0,1,397852.5,9502517.5,0,0.0
1,1,397852.5,9502502.5,0,0.0
2,1,397852.5,9502487.5,0,0.0
3,1,397852.5,9502472.5,0,0.0
4,1,397852.5,9502457.5,0,0.0


In [37]:
df_ASTER.describe()

,band,E_utm,N_utm,spatial_ref,band_data
count,2.825928e+07,2.825928e+07,2.825928e+07,28259280.0,2.825928e+07
mean,7.500000e+00,4.098600e+05,9.493075e+06,0.0,5.556849e+02
std,4.031129e+00,6.936862e+03,5.455958e+03,0.0,6.729804e+02
min,1.000000e+00,3.978525e+05,9.483632e+06,0.0,0.000000e+00
25%,4.000000e+00,4.038525e+05,9.488354e+06,0.0,3.200000e+01
50%,7.500000e+00,4.098600e+05,9.493075e+06,0.0,6.900000e+01
75%,1.100000e+01,4.158675e+05,9.497796e+06,0.0,1.306000e+03
max,1.400000e+01,4.218675e+05,9.502518e+06,0.0,1.941000e+03


## SENSORES

#### Computing NDVI

In [38]:
# Ndvi - 1984
NDVI = np.array(((df_rs.ASTER_NIR_1984 - df_rs.ASTER_RED_1984) + 1)/((df_rs.ASTER_NIR_1984 + df_rs.ASTER_RED_1984) + 1))
df_rs['NDVI_1984'] = (NDVI - NDVI.mean())/NDVI.std()

# Ndvi - 1986
NDVI = np.array(((df_rs.ASTER_NIR_1986 - df_rs.ASTER_RED_1986) + 1)/((df_rs.ASTER_NIR_1986 + df_rs.ASTER_RED_1986) + 1))
df_rs['NDVI_1986'] = (NDVI - NDVI.mean())/NDVI.std()

# Ndvi - 2004-2005
NDVI = np.array(((df_rs.NIR - df_rs.RED) + 1)/((df_rs.NIR + df_rs.RED) f/AST/stq_AST_13-05-+ 1))
df_rs['NDVI_2004'] = (NDVI - NDVI.mean())/NDVI.std()
# Ndvi - 2021
NDVI = np.array(((df_rs.OLI_NIR - df_rs.OLI_RED) + 1)/((df_rs.OLI_NIR + df_rs.OLI_RED) + 1))
df_rs['NDVI_2021'] = (NDVI - NDVI.mean())/NDVI.std()

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (1441942782.py, line 10)

### Pixel size

In [ ]:
df_result

In [ ]:
df_rs = df_result.rename(columns={'X':'E_utm','Y':'N_utm'})
df_rs

In [ ]:
xpixel_size = (df_rs.E_utm.max()-df_rs.E_utm.min())/df_rs.E_utm.unique().size
ypixel_size = (df_rs.N_utm.max()-df_rs.N_utm.min())/df_rs.N_utm.unique().size
print('x:', xpixel_size, 'y:', ypixel_size)

In [ ]:
nx=df_rs.E_utm.unique().size
ny=df_rs.N_utm.unique().size
ratio=ny/nx
xs = df_rs.E_utm.values.reshape(ny, nx)
ys = df_rs.N_utm.values.reshape(ny, nx)

In [ ]:
var = np.array((df_rs[0]. values+1)/(df_rs[1].values+1)).reshape(ny, nx)
plt.figure(figsize=(12, int(ratio * 10)))
plt.pcolormesh(xs, ys, var, cmap='jet', shading='auto', vmin=np.percentile(var, 15), vmax=np.percentile(var, 85))
plt.colorbar(label='ratio')

In [ ]:
features = list(df_rs.columns[2:])
print(features)

In [ ]:
df_rs= df_rs.drop([31,15],axis=1)
df_rs

In [ ]:
print(df_rs[15].mean())
print(df_rs['MDT'].mean())

In [ ]:
df_rs = df_rs.fillna(-99999)

In [ ]:
plot_corr(df_rs[features], size=45)
plt.savefig('figs/correlation_matrix.png', dpi=400, bbox_inches='tight')

### PCA

In [ ]:
features = ['GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2_1', 'SWIR2_2', 'SWIR2_3',
            'SWIR2_4', 'SWIR2_5', 'TIR1', 'TIR2', 'TIR3', 'DEM', 'Slope', 
            'NDVI_1984', 'NDVI_1986', 'NDVI_2004', 'NDVI_2021']

In [ ]:
features = list(df_rs.columns[2:])
print(features)

In [ ]:
pcs = compute_pca(df_rs, features)
#plt.savefig('figs/eigenvalues_matrix.png', dpi=400, bbox_inches='tight')

In [ ]:
def pcs_dataframe(pcs, x, y, n_pcs=pcs.shape[0]):
    
    data = {'X':x, 'Y':y}
    
    components = np.arange(1, n_pcs+1)
    pc = ['PC']*(len(components)+1)

    PCs = []
    for i, k in zip(pc, components):
        PCs.append(i+str(k))

    for n in range(n_pcs):
        data[PCs[n]] = pcs[n][:,0]

    df = pd.DataFrame(data)

    return df

In [ ]:
pcs_df=pcs_dataframe(pcs, df_rs.E_utm, df_rs.N_utm)
pcs_df.head(3)

In [ ]:
pcs_df.to_csv('RS_PCs.xyz', sep=',', index_label=False, index=False)

## Ternary plots

In [ ]:
# defining arguments
variables_rgb = ['PC1', 'PC2', 'PC3'] # a list with the name of the variables in the RGB order;

In [ ]:
RGB, extent = ternary.grid(pcs_df, variables_rgb, 'X', 'Y', center=0.5, stretch=0.6)

# the first output consists of the grid (M,N 3) used to create the RGB image.
# the second output consists of the extent (coordinates) of the espacial data we intend to plot.

In [ ]:
ax = ternary.plot(RGB, variables_rgb, extent, triangle_position=[0.92, 0.2, 0.2, 0.2])
ax.grid(linewidth=0.9, alpha=0.8)
plt.savefig(f'figs/pca_ternary_{variables_rgb[0]+variables_rgb[1]+variables_rgb[2]}.png',
            dpi=400, bbox_inches='tight')

In [ ]:
# defining arguments
# a list with the name of the variables in the RGB order;
variables_rgb = ['PC1', 'PC2', 'PC3'] 

RGB, extent = ternary.grid(df_rs, variables_rgb, 'E_utm', 'N_utm', center=0.5, stretch=.5)

In [ ]:
# the first output consists of the grid (M,N 3) used to create the RGB image.
# the second output consists of the extent (coordinates) of the espacial data we intend to plot.

ax = ternary.plot(RGB, variables_rgb, extent, triangle_position=[0.82, 0.2, 0.2, 0.2])
ax.grid(linewidth=0.9, alpha=0.8)

### Self-organizing maps (SOM)

In [ ]:
from sklearn_som.som import SOM

In [ ]:
#features = ['GREEN', 'DEM', 'SWIR1', 'NDVI_1984', 'NDVI_1986']
#print(features)

In [ ]:
features = list(df_rs.columns[2:])
print(features)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data = scaler.fit_transform(df_rs[features].values)
# data = df_rs[features].values

In [ ]:
n_clusters=7
lito_SOM = SOM(m=n_clusters, n=1, sigma=1.25, dim=len(features), max_iter=1000)
lito_SOM.fit(data)

In [ ]:
# predição de classes
predictions = lito_SOM.predict(data)

In [ ]:
# create labels
cluster_labels=[]
for i in range(n_clusters):
    cluster_labels+=[f'Classe {i+1}']

In [ ]:
# classes weights
fig, ax = plt.subplots()
im=ax.matshow(lito_SOM.weights)
for (i, j), z in np.ndenumerate(lito_SOM.weights):
    ax.text(j, i, '{:0.2f}'.format(z), ha='center', va='center')

plt.yticks(range(n_clusters), cluster_labels, fontsize=9)
plt.xticks(range(len(features)), features, rotation=55, fontsize=10, ha='left')
fig.colorbar(im, label='weights', orientation='horizontal')
plt.gca().set_aspect('equal')
plt.gcf().set_size_inches(5, 6)
plt.show()

In [ ]:
import matplotlib
id_ = [1, 2, 3, 4, 5,6,7]
relcolor =  matplotlib.cm.Set3
colors = np.array(relcolor.colors)[id_]
relcolor = matplotlib.colors.ListedColormap(colors)

In [ ]:
idxs=np.arange(0, n_clusters, 1)
half=(idxs[1]-idxs[0])/2
ticks=np.linspace(idxs[0]+half, idxs[-1]-half, n_clusters)

In [ ]:
fig, ax=plt.subplots(figsize=(11, int(ratio*11)), facecolor='w')
im=plt.pcolormesh(xs, ys, predictions.reshape(ny, nx), cmap=relcolor, shading='auto')
plt.xlim(xs.min(), xs.max())
plt.ylim(ys.min(), ys.max())
cbar_ax = fig.add_axes([0.93, 0.3, 0.05, 0.4])
cbar = fig.colorbar(im, cax=cbar_ax, label = u'Classes', orientation='vertical',
                    ticks=ticks)
cbar.ax.set_yticklabels(cluster_labels, fontsize=8)
plt.show()

### Hillshade

In [ ]:
hillshd = hillshade(df_rs['DEM'].values.reshape(ny, nx), 330, 15)

In [ ]:
fig, ax=plt.subplots(figsize=(12, int(ratio*12)), facecolor='w')
slope = df_rs.Slope.values.reshape(ny, nx)
backg = ax.pcolormesh(xs, ys, hillshd, vmin=np.percentile(hillshd, 15), vmax=np.percentile(hillshd, 85),
                      shading='auto', cmap = 'Greys', alpha=1.0)
im=plt.pcolormesh(xs, ys, predictions.reshape(ny,nx), cmap=relcolor, shading='auto', alpha=0.75)

front = ax.pcolormesh(xs, ys, slope, vmin=np.percentile(slope, 15), vmax=np.percentile(slope, 85),
                      shading='auto', cmap = 'Greys', alpha=0.1)
plt.xlim(xs.min(), xs.max())
plt.ylim(ys.min(), ys.max())
cbar_ax = fig.add_axes([0.93, 0.3, 0.05, 0.4])
cbar = fig.colorbar(im, cax=cbar_ax, label = u'Classes', orientation='vertical',
                    ticks=ticks)
cbar.ax.set_yticklabels(cluster_labels,fontsize=7)